<a href="https://colab.research.google.com/github/richmondvan/melanoma-detection/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training

Import all modules and mount Google Drive

In [ ]:
# Must be run every time!

from pathlib import Path # Manage file paths
import pickle # Storing epoch number
from google.colab import drive # For mounting GDrive
import matplotlib.pyplot as plt # For data preview
import os
import tempfile

# tensorflow imports
from tensorflow.keras import metrics, regularizers, models # utilities
from tensorflow.keras.optimizers import Adam # optimizer
from tensorflow.keras.losses import BinaryCrossentropy # loss function
from tensorflow.keras.applications import MobileNetV2 # base model for transfer learning
from tensorflow.keras.models import Sequential # model architecture
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D # additional layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Data pipeline
from tensorflow.keras.callbacks import CSVLogger # CSV logger callback

# Mount Google Drive
drive.mount('/content/gdrive') # Mount the google drive where previous weights are stored

!git clone https://github.com/richmondvan/isic-image-database.git # clone the dataset from Github

Prepare datasets

In [ ]:
# Setting up file paths
PATH = "/content/isic-image-database/"

# Filepaths for each of the datasets
TRAINING_PATH = Path(PATH + "training/")
VALIDATION_PATH = Path(PATH + "validation/")
TEST_PATH = Path(PATH + "test/")

# Create image generators for training data
train_image_generator = ImageDataGenerator(
    rescale=1./255, # Convert RGB values into floats between 0 and 1
    brightness_range=(0.90, 1.10), # For data augmentation: Increase or decrease brightness by at most 10%.
    zoom_range=[0.9, 1], # For data augmentation: Zoom in by at most 10%. No zooming out.
    horizontal_flip=True, # For data augmentation: Flip along horizontal axis.
    vertical_flip=True) # For data augmentation: Flip along vertical axis. 

# Create image generators for validation data
validation_image_generator = ImageDataGenerator(rescale=1./255) # No data augmentation, validation dataset must remain consistent.

# Some constants
BATCH_SIZE = 32 # Small batch size for good generalization

# Image is a square, 224x224.
IMG_HEIGHT = 224 
IMG_WIDTH = IMG_HEIGHT 

# Get size of data generators, by globbing based on file extension .jpg
TRAIN_LEN = len(list(TRAINING_PATH.glob("*/*.jpg")))
VALID_LEN = len(list(VALIDATION_PATH.glob("*/*.jpg")))

# Hardcoded directory names to sort classes from.
CLASS_NAMES = ['benign', 'malignant']

# Get generated datasets
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=TRAINING_PATH,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary',
                                                           classes=CLASS_NAMES)

val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=VALIDATION_PATH,
                                                              shuffle=True,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary',
                                                              classes=CLASS_NAMES)

Show images

In [ ]:
image_batch, label_batch = next(train_data_gen)

def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

show_batch(image_batch, label_batch)

Prepare metrics and weights

In [ ]:
# Get some training weights to offset class imbalance
numBenign = len(list(TRAINING_PATH.glob("benign/*.jpg")))
numMalignant = len(list(TRAINING_PATH.glob("malignant/*.jpg")))
total = numBenign + numMalignant

# Depending on how we value precision and recall, this may change in the future...
ADDITIONAL_WEIGHT_MULTIPLIER = 1.0

# Weights for learning. Malignant cases are weighted more heavily, to offset imbalanced datasets.
weight_for_0 = (1 / numBenign) * (total) / 2.0 
weight_for_1 = (ADDITIONAL_WEIGHT_MULTIPLIER / numMalignant) * (total) / 2.0
class_weight = {0: weight_for_0, 1: weight_for_1}

# Check the weight.
print(class_weight)

# Metrics we will be using to assess accuracy
METRICS = [
      metrics.BinaryAccuracy(name='acc'), # Raw accuracy.
      metrics.TruePositives(name='tp'),
      metrics.FalsePositives(name='fp'),
      metrics.TrueNegatives(name='tn'),
      metrics.FalseNegatives(name='fn'), 
      metrics.Precision(name='pre'), # tp / (tp + fp)
      metrics.Recall(name='rec'), # tp / (tp + fn)
      metrics.AUC(name='auc'), # area under curve
]

Prepare model

In [ ]:
# Hyperparameters
REG_LAMBDA = 0.001 # coefficient for regularization
DROPOUT = 0.1 # 10% dropout per layer
ACTIVATION = "relu"
NEURONS_PER_LAYER = 512
NUM_DENSE_LAYERS = 8

# The image shape has three channels, so it is 224x224x3.
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

# Get the base model for transfer learning, but remove the classification head. Freeze.
base_model = MobileNetV2(input_shape = IMG_SHAPE, include_top=False, weights='imagenet', alpha=1.4, pooling='avg')
base_model.trainable = False

# Build the full Sequential model.
model = Sequential()
model.add(base_model)
# Add a base dropout
model.add(Dropout(DROPOUT))
# Add some dense layers.
for x in range(NUM_DENSE_LAYERS):
    model.add(Dense(NEURONS_PER_LAYER, kernel_regularizer=regularizers.l2(REG_LAMBDA), activation=ACTIVATION))
    model.add(Dropout(DROPOUT))
# Add a classification head.
model.add(Dense(1, activation="sigmoid"))

# Small learning rate.
LEARNING_RATE = 0.0005

def compileModel(learningRate):
    global model
    model.compile(
        optimizer=Adam(learning_rate=learningRate),
        loss=BinaryCrossentropy(from_logits=True),
        metrics=METRICS)

compileModel(LEARNING_RATE)
model.summary()

Load model weights and last epoch

In [ ]:
# Get last epoch number from pickled file

MODEL_FILEPATH = f"/content/gdrive/My Drive/MelanomaDetectionModels/{NEURONS_PER_LAYER}_{NUM_DENSE_LAYERS}/"
EPOCH_FILEPATH = MODEL_FILEPATH + "epochnum.pkl"

# Loads weights.
def loadWeights():
    global epoch, model, EPOCH_FILEPATH, MODEL_FILEPATH
    try: 
        infile = open(EPOCH_FILEPATH, 'rb')
        infile.seek(0)
        epoch = pickle.load(infile)
        try:
            model.load_weights(MODEL_FILEPATH + f"epoch{epoch}.h5")
            infile.close()
        except:
            pass
    except: 
        # Otherwise start again (only happens if no epoch number found)
        epoch = 0
    print(epoch)

loadWeights()

Prepare CSV logger

In [ ]:
# File where we store our CSV history

HISTORY_FILEPATH = MODEL_FILEPATH + "history.csv"

csv_logger = CSVLogger(HISTORY_FILEPATH, append=True)

Train model

In [ ]:
# Train for 150 epochs

epochsToTrain = 150

def trainModel(epochsToFit): # Trains the model up to a specified epoch number
    global epoch, model, train_data_gen, val_data_gen, VALID_LEN, TRAIN_LEN, BATCH_SIZE, class_weight, csv_logger, MODEL_FILEPATH, EPOCH_FILEPATH
    if epoch < epochsToFit:
        for i in range(epoch, epochsToFit):
            # Train the model, one epoch at a time.
            history = model.fit(x=train_data_gen, # training data
                                epochs=i+1, # epoch to train to (next epoch)
                                initial_epoch=i, # current epoch
                                verbose=1, # show progress
                                validation_data=val_data_gen, # validation data 
                                validation_steps=VALID_LEN // BATCH_SIZE, # validation batch steps
                                steps_per_epoch=TRAIN_LEN // BATCH_SIZE, # training batch steps
                                class_weight=class_weight, # class weights (to avoid imbalanced training)
                                callbacks = [csv_logger]) # CSV logger callback
            model.save_weights(MODEL_FILEPATH + f"epoch{i + 1}.h5") # save the new epoch weights for records later.
            outfile = open(EPOCH_FILEPATH, 'wb') # dump the current epoch (in case Colab decides to stop before the loop finishes.)
            pickle.dump(i+1, outfile)
            outfile.close()

trainModel(epochsToTrain)

Fine-Tune training, on a step-wise basis.

In [ ]:
BASE_MODEL_SIZE = len(base_model.layers) # Number of layers in MobileNetV2 (should be 156)
EPOCHS_FOR_FINE_TUNE = 100 # Epochs to fine-tune for at each stage.

CURRENT_CYCLE = 1 # Default cycle number

NUM_LAYERS_TO_FINE_TUNE = 25 # Number of layers that are fine-tuned at once.

if epoch >= epochsToTrain:
    loadWeights() # Load weights (to update epoch num)
    CURRENT_CYCLE = 1 + int((epoch - epochsToTrain) / EPOCHS_FOR_FINE_TUNE) # calculate current cycle from weights by rounding down quotient.

FINE_TUNE_CYCLES = 2 # Total number of cycles

print(CURRENT_CYCLE)

regularizer = regularizers.l2(REG_LAMBDA) # Regularization for fine-tuning

for cycleIndex in range(CURRENT_CYCLE, FINE_TUNE_CYCLES + 1): # loops through all steps of fine-tuning from current to total.
    fine_tune_from = BASE_MODEL_SIZE - 1 - (NUM_LAYERS_TO_FINE_TUNE * cycleIndex) # Calculates which layer fine-tuning begins, minus one for the pooling layer.
    base_model.trainable = False # Set everything to untrainable at first...
    for layer in base_model.layers[fine_tune_from:]: # and then set all layers after fine_tune_from to be trainable.
        layer.trainable = True    
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)
    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)

    compileModel(LEARNING_RATE/10) # compile the new model with a reduced learning-rate
    model.summary() # show the model
    trainModel(epochsToTrain + EPOCHS_FOR_FINE_TUNE * cycleIndex) # train it up to the new epoch
    loadWeights() # load new epoch.

In [ ]:
# Reset epochnum

# import pickle

# MODEL_FILEPATH = f"/content/gdrive/My Drive/MelanomaDetectionModels/512_8/"
# EPOCH_FILEPATH = MODEL_FILEPATH + "epochnum.pkl"
# outfile = open(EPOCH_FILEPATH, 'wb') # dump the current epoch (in case Colab decides to stop before the loop finishes.)
# pickle.dump(150, outfile)
# outfile.close()